In [ ]:
import pandas as pd
from openai import OpenAI
import re
import time
import pickle
import re

# Set up

In [ ]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [ ]:
def get_text_between_quotes(input_string):
    pattern = r'"([^"]*)"'
    matches = re.findall(pattern, input_string)
    try:
        return matches[0]
    except Exception as e:
        return input_string

In [ ]:
with open('./dataset/full_formal_script_temp0.8.pkl', 'rb') as fp:
    data_formal = pickle.load(fp)

with open('./dataset/full_informal_script_temp0.8.pkl', 'rb') as fp:
    data_informal = pickle.load(fp)

with open('./dataset/full_novel_script_temp0.8.pkl', 'rb') as fp:
    data_novel = pickle.load(fp)

df = pd.read_csv('./dataset/raw.csv', index_col='uid')

In [ ]:
data_novel

# Load model

## Load finetuned model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

In [ ]:
model_path = "./Tuning/checkpoint/Phi-3-mini-4k-instruct"
token_path = f"{model_path}/tokenize"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(token_path)

In [ ]:
peft_config = PeftConfig.from_pretrained(model_path)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
model = PeftModel.from_pretrained(base_model, model_path)

In [ ]:
model = model.merge_and_unload()

In [ ]:
model.eval()

In [ ]:
def generate_text(prompt, max_length=400, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=temperature,
            do_sample=True,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def get_dict(text):
    res = None
    try:
        res = re.findall('{.*}', text)[0]
        res = eval(res)
    except:
        print(f"Error: {text}")
    return res

In [ ]:
prompt = """<|system|>
You are a helpful, smart, kind, and efficient AI assistant. You always fulfill the user's requests to the best of your ability.<|end|>
<|user|>
Extract name, age, job from the sentence to json format. if the information doesn't exist fill null.

Sentence: 'John Doe is a 35-year-old software engineer with a passion for artificial intelligence.'<|end|>
<|assistant|>
"""

generated_text_low_temp = generate_text(prompt, temperature=0.1)
get_dict(generated_text_low_temp)

# Predict

In [ ]:
def call_llm(data):
    promtp = """Extract name, age, job from the sentence to json format. 
    if the information doesn't exsits fill null.
    
    sentence: '{}'
    """
    
    start_time = time.time()
    completion = client.chat.completions.create(
      model="microsoft/Phi-3-mini-4k-instruct-gguf",
      messages=[
        {"role": "system", "content": "You are a helpful, smart, kind, and efficient AI assistant. You always fulfill the user's requests to the best of your ability."},
        {"role": "user", "content": promtp.format(data)}
      ],
      temperature=0,
    )
    end_time = time.time()
    content = completion.choices[0].message.content
    return end_time-start_time, content.strip().replace('null', '\"\"')

def call_llm_model(data):
    prompt = f"""<|system|>
You are a helpful, smart, kind, and efficient AI assistant. You always fulfill the user's requests to the best of your ability.<|end|>
<|user|>
Extract name, age, job from the sentence to json format. if the information doesn't exist fill null.

Sentence: '{data}'<|end|>
<|assistant|>"""
    start_time = time.time()
    generated_text_low_temp = generate_text(prompt, temperature=0.1)
    end_time = time.time()
    return end_time-start_time, get_dict(generated_text_low_temp)

In [ ]:
# Predict formal data
predict_data = {}
for temp in data_formal:
    for id in data_formal[temp]:
        # _, result = call_llm(data_formal[temp][id]) # Before finetune
        _, result = call_llm_model(data_formal[temp][id]) # After finetune
        try:
            if not isinstance(result, dict):
                predict_data[id] = eval(result)
            else:
                predict_data[id] = result
        except:
            print(result)

# with open('./dataset/predict_formal_script.pkl', 'wb') as fp:
#     pickle.dump(predict_data, fp)
with open('./dataset/finetune_predict_formal_script.pkl', 'wb') as fp:
    pickle.dump(predict_data, fp)

In [ ]:
# Predict informal data
predict_data = {}
for temp in data_informal:
    for id in data_informal[temp]:
        # _, result = call_llm(data_informal[temp][id]) # Before finetune
        _, result = call_llm_model(data_informal[temp][id]) # After finetune
        try:
            if not isinstance(result, dict):
                predict_data[id] = eval(result)
            else:
                predict_data[id] = result
        except:
            print(result)

# with open('./dataset/predict_informal_script.pkl', 'wb') as fp:
#     pickle.dump(predict_data, fp)
with open('./dataset/finetune_predict_informal_script.pkl', 'wb') as fp:
    pickle.dump(predict_data, fp)

In [ ]:
# Predict novel data
predict_data = {}
for temp in data_novel:
    for id in data_novel[temp]:
        # _, result = call_llm(data_novel[temp][id]) # Before finetune
        _, result = call_llm_model(data_novel[temp][id]) # After finetune
        try:
            if not isinstance(result, dict):
                predict_data[id] = eval(result)
            else:
                predict_data[id] = result
        except:
            print(result)

# with open('./dataset/predict_novel_script.pkl', 'wb') as fp:
#     pickle.dump(predict_data, fp)
with open('./dataset/finetune_predict_novel_script.pkl', 'wb') as fp:
    pickle.dump(predict_data, fp)

# Evaluate

## Calculate score

In [ ]:
with open('./dataset/predict_formal_script.pkl', 'rb') as fp:
    predict_formal = pickle.load(fp)

with open('./dataset/predict_informal_script.pkl', 'rb') as fp:
    predict_informal = pickle.load(fp)

with open('./dataset/predict_novel_script.pkl', 'rb') as fp:
    predict_novel = pickle.load(fp)

with open('./dataset/finetune_predict_formal_script.pkl', 'rb') as fp:
    finetune_predict_formal = pickle.load(fp)

with open('./dataset/finetune_predict_informal_script.pkl', 'rb') as fp:
    finetune_predict_informal = pickle.load(fp)

with open('./dataset/finetune_predict_novel_script.pkl', 'rb') as fp:
    finetune_predict_novel = pickle.load(fp)

with open('./dataset/full_formal_script_temp0.8.pkl', 'rb') as fp:
    data_formal = pickle.load(fp)

with open('./dataset/full_informal_script_temp0.8.pkl', 'rb') as fp:
    data_informal = pickle.load(fp)

with open('./dataset/full_novel_script_temp0.8.pkl', 'rb') as fp:
    data_novel = pickle.load(fp)

df = pd.read_csv('./dataset/raw.csv', index_col='uid')

In [ ]:
datasets = ['formal', 'informal', 'novel']
features = ['name', 'age', 'job']
for dataset in datasets:
    for feature in features:
        key = f"pred_{dataset}_{feature}"
        key = f"finetune_pred_{dataset}_{feature}"
        df[key] = None

## Evaluate finetune model

In [ ]:
for i in finetune_predict_formal:
    dataset = 'formal'
    for feature in features:
        key = f"finetune_pred_{dataset}_{feature}"
        try:
            if isinstance(finetune_predict_formal[i][feature], list):
                df.loc[i, key] =  " ".join(finetune_predict_formal[i][feature])
            else:
                df.loc[i, key] =  finetune_predict_formal[i][feature]
        except Exception as e:
            print(f"{e}")

for i in finetune_predict_informal:
    dataset = 'informal'
    for feature in features:
        key = f"finetune_pred_{dataset}_{feature}"
        try:
            if isinstance(finetune_predict_informal[i][feature], list):
                df.loc[i, key] =  " ".join(finetune_predict_informal[i][feature])
            else:
                df.loc[i, key] =  finetune_predict_informal[i][feature]
        except Exception as e:
            print(f"{e}")

for i in finetune_predict_novel:
    dataset = 'novel'
    for feature in features:
        key = f"finetune_pred_{dataset}_{feature}"
        try:
            if isinstance(finetune_predict_novel[i][feature], list):
                df.loc[i, key] =  " ".join(finetune_predict_novel[i][feature])
            else:
                df.loc[i, key] =  finetune_predict_novel[i][feature]
        except Exception as e:
            print(f"{e}")

## Evaluate base model

In [ ]:
for i in predict_formal:
    dataset = 'formal'
    for feature in features:
        key = f"pred_{dataset}_{feature}"
        try:
            if isinstance(predict_formal[i][feature], list):
                df.loc[i, key] =  " ".join(predict_formal[i][feature])
            else:
                df.loc[i, key] =  predict_formal[i][feature]
        except Exception as e:
            print(f"{e}")

for i in predict_informal:
    dataset = 'informal'
    for feature in features:
        key = f"pred_{dataset}_{feature}"
        try:
            if isinstance(predict_informal[i][feature], list):
                df.loc[i, key] =  " ".join(predict_informal[i][feature])
            else:
                df.loc[i, key] =  predict_informal[i][feature]
        except Exception as e:
            print(f"{e}")

for i in predict_novel:
    dataset = 'novel'
    for feature in features:
        key = f"pred_{dataset}_{feature}"
        try:
            if isinstance(predict_novel[i][feature], list):
                df.loc[i, key] =  " ".join(predict_novel[i][feature])
            else:
                df.loc[i, key] =  predict_novel[i][feature]
        except Exception as e:
            print(f"{e}")

In [ ]:
for dataset in datasets:
    for feature in features:
        key = f"pred_{dataset}_{feature}"
        score_key = f"pred_{dataset}_{feature}_score"
        df[score_key] = df[feature] == df[key]
        
        key = f"finetune_pred_{dataset}_{feature}"
        score_key = f"finetune_pred_{dataset}_{feature}_score"
        df[score_key] = df[feature] == df[key]

## Calculate all dataset

In [ ]:
lenght = len(df)
for dataset in datasets:
    print(f"-------- {dataset.capitalize()} --------")
    for feature in features:
        score_key = f"pred_{dataset}_{feature}_score"
        print(f"{feature.capitalize()} accuracy: {sum(df[score_key]) / len(df)}")
    print(f"========================================\n")


for dataset in datasets:
    print(f"-------- Finetune - {dataset.capitalize()} --------")
    for feature in features:
        score_key = f"finetune_pred_{dataset}_{feature}_score"
        print(f"{feature.capitalize()} accuracy: {sum(df[score_key]) / len(df)}")
    print(f"========================================\n")

In [ ]:
# -------- Formal --------
# Name accuracy: 0.8715
# Age accuracy: 0.959
# Job accuracy: 0.612
# ========================================

# -------- Informal --------
# Name accuracy: 0.8985
# Age accuracy: 0.989
# Job accuracy: 0.686
# ========================================

# -------- Novel --------
# Name accuracy: 0.882
# Age accuracy: 0.95
# Job accuracy: 0.5505
# ========================================

# -------- Finetune - Formal --------
# Name accuracy: 0.984
# Age accuracy: 0.984
# Job accuracy: 0.9995
# ========================================

# -------- Finetune - Informal --------
# Name accuracy: 0.9775
# Age accuracy: 0.9985
# Job accuracy: 1.0
# ========================================

# -------- Finetune - Novel --------
# Name accuracy: 0.949
# Age accuracy: 0.9595
# Job accuracy: 0.9885
# ========================================

## Calculate only test dataset

In [ ]:
lenght = len(df)
test_df = df[1800:]
for dataset in datasets:
    print(f"-------- Test dataset - {dataset.capitalize()} --------")
    for feature in features:
        score_key = f"pred_{dataset}_{feature}_score"
        print(f"{feature.capitalize()} accuracy: {sum(test_df[score_key]) / len(test_df)}")
    print(f"========================================\n")


for dataset in datasets:
    print(f"-------- Finetune test dataset - {dataset.capitalize()} --------")
    for feature in features:
        score_key = f"finetune_pred_{dataset}_{feature}_score"
        print(f"{feature.capitalize()} accuracy: {sum(test_df[score_key]) / len(test_df)}")
    print(f"========================================\n")

In [ ]:
# -------- Test dataset - Formal --------
# Name accuracy: 0.87
# Age accuracy: 0.965
# Job accuracy: 0.685
# ========================================

# -------- Test dataset - Informal --------
# Name accuracy: 0.895
# Age accuracy: 0.98
# Job accuracy: 0.75
# ========================================

# -------- Test dataset - Novel --------
# Name accuracy: 0.875
# Age accuracy: 0.935
# Job accuracy: 0.55
# ========================================

# -------- Finetune test dataset - Formal --------
# Name accuracy: 0.965
# Age accuracy: 0.975
# Job accuracy: 1.0
# ========================================

# -------- Finetune test dataset - Informal --------
# Name accuracy: 0.96
# Age accuracy: 0.995
# Job accuracy: 1.0
# ========================================

# -------- Finetune test dataset - Novel --------
# Name accuracy: 0.925
# Age accuracy: 0.93
# Job accuracy: 0.955
# ========================================